In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler,LabelEncoder
import torch
from torch.utils.data import Dataset, DataLoader, Subset, random_split
from torchvision import transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as f
from PIL import Image
import os
from sklearn import metrics
import glob
from torchtsmixer import TSMixer



In [2]:
df=pd.read_csv("data_imu_birlestirilmis.csv")

In [3]:
label_encoder=LabelEncoder()

In [4]:
df["Class Name"]=label_encoder.fit_transform(df["Class Name"])

In [5]:
print(label_encoder.classes_)



['falling' 'not_falling']


In [6]:
df

,Unnamed: 0,ax,ay,az,gx,gy,gz,timestamp,Class Name
0,0,1.091892,8.707838,1.765225,-0.036396,0.063694,-0.036396,1900-01-01 17:44:42.000000,1
1,1,1.083568,8.732087,1.765318,-0.036781,0.064346,-0.036686,1900-01-01 17:44:42.020000,1
2,2,1.075247,8.756325,1.765411,-0.037166,0.064996,-0.036976,1900-01-01 17:44:42.040000,1
3,3,1.066932,8.780539,1.765503,-0.037551,0.065642,-0.037265,1900-01-01 17:44:42.060000,1
4,4,1.058625,8.804720,1.765596,-0.037936,0.066281,-0.037553,1900-01-01 17:44:42.080000,1
...,...,...,...,...,...,...,...,...,...
51645,51645,1.611913,8.047413,-3.202833,-0.106623,0.084735,-0.050371,1900-01-01 17:59:15.899794,1
51646,51646,1.626502,8.043946,-3.201882,-0.118920,0.123886,-0.071806,1900-01-01 17:59:15.919794,1
51647,51647,1.641042,8.040466,-3.200937,-0.131261,0.163164,-0.093307,1900-01-01 17:59:15.939794,1
51648,51648,1.655544,8.036974,-3.199998,-0.143635,0.202538,-0.114858,1900-01-01 17:59:15.959793,1


In [7]:
dataframe=df

In [8]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51650 entries, 0 to 51649
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  51650 non-null  int64  
 1   ax          51650 non-null  float64
 2   ay          51650 non-null  float64
 3   az          51650 non-null  float64
 4   gx          51650 non-null  float64
 5   gy          51650 non-null  float64
 6   gz          51650 non-null  float64
 7   timestamp   51650 non-null  object 
 8   Class Name  51650 non-null  int32  
dtypes: float64(6), int32(1), int64(1), object(1)
memory usage: 3.3+ MB



Veriler üzerinde standartlaştırma yapmak istiyorsanız, verileri modelinize vermeden önce StandardScaler gibi bir ölçekleyici ile işleyebilirsiniz. İşte nasıl ekleyebileceğinizin bir örneği:

Standartlaştırmayı Eklemek İçin:

StandardScaler kullanılarak her bir ax, ay, az, gx, gy, gz sütunu standartlaştırılıyor (ortalaması 0, varyansı 1 olacak şekilde).
Bu işlemi __getitem__ fonksiyonu yerine, __init__ fonksiyonunda tüm veri setine uygulayarak yapıyoruz. Bu sayede veriler model eğitimi öncesinde ölçeklendirilmiş oluyor.
Standartlaştırma, modelin daha hızlı ve kararlı bir şekilde öğrenmesine yardımcı olabilir, çünkü girdi verilerindeki farklı büyüklüklerdeki özelliklerin etkilerini dengeleyerek modelin dikkatini önemli desenlere yönlendirir.

import torch
from torch.utils.data import Dataset
from scipy import stats

class CustomDataset(Dataset):
    def __init__(self, dataframe, sequence_length=25):
        self.dataframe = dataframe
        self.features = ["ax", "ay", "az", "gx", "gy", "gz"]
        self.labels = "Class Name"
        self.sequence_length = sequence_length

        self.scaler = StandardScaler()
        self.dataframe[self.features] = self.scaler.fit_transform(self.dataframe[self.features])

    def __len__(self):
        return len(self.dataframe) - self.sequence_length + 1

    def __getitem__(self, idx):
         
        features_sequence = self.dataframe.loc[idx:idx + self.sequence_length - 1, self.features].values.astype(float)
        
        
        label = self.dataframe.loc[idx + self.sequence_length - 1, self.labels]
        
         
        features_tensor = torch.tensor(features_sequence, dtype=torch.float32)
        label_tensor = torch.tensor(label, dtype=torch.long)

        return features_tensor, label_tensor

    

In [9]:
import torch
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler

class TSMixerDataset(Dataset):
    
    def __init__(self, dataframe, sequence_length=25):
        self.dataframe = dataframe
        self.features = ["ax", "ay", "az", "gx", "gy", "gz"]
        self.labels = "Class Name"
        self.sequence_length = sequence_length

        self.scaler = StandardScaler()
        self.dataframe[self.features] = self.scaler.fit_transform(self.dataframe[self.features])
    
    def __len__(self):
        return len(self.dataframe) - self.sequence_length + 1
    
    def __getitem__(self, idx):
        while True:
            features_sequence = self.dataframe.loc[idx:idx + self.sequence_length - 1, self.features].values.astype(float)
            labels_sequence = self.dataframe.loc[idx:idx + self.sequence_length - 1, self.labels].values

            # Tüm etiketlerin aynı olup olmadığını kontrol et
            if len(set(labels_sequence)) == 1:  # Tüm etiketler aynı
                label = labels_sequence[0]
                break  # Aynı etiket bulundu, döngüden çık

            # Farklı etiketler varsa, bir sonraki indeksi kontrol et
            idx += 1
            if idx + self.sequence_length > len(self.dataframe):  # Sınır kontrolü
                raise IndexError("Veri setinde uygun pencere kalmadı.")

        features_tensor = torch.tensor(features_sequence, dtype=torch.float32)  # (sequence_length, features_count)
        label_tensor = torch.tensor(label, dtype=torch.long)

        return features_tensor, label_tensor


In [10]:
dataset = TSMixerDataset(dataframe)

 
labels = dataset.dataframe['Class Name'].values[:len(dataset)]   

 
indices = list(range(len(dataset)))

 
train_indices, test_indices = train_test_split(indices, test_size=0.2, stratify=labels)
 
train_dataset = torch.utils.data.Subset(dataset, train_indices)
test_dataset = torch.utils.data.Subset(dataset, test_indices)

 
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [11]:
print(f"Train dataset size: {len(train_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

Train dataset size: 41300
Test dataset size: 10326


In [12]:
train_dataset[6]

(tensor([[-0.1973,  1.4236, -0.3150,  0.0243,  0.1950,  1.7694],
         [-0.1946,  1.4154, -0.3331,  0.0300,  0.1794,  1.7647],
         [-0.1915,  1.4055, -0.3514,  0.0356,  0.1629,  1.7583],
         [-0.1877,  1.3941, -0.3697,  0.0412,  0.1455,  1.7501],
         [-0.1835,  1.3811, -0.3880,  0.0468,  0.1273,  1.7402],
         [-0.1787,  1.3667, -0.4063,  0.0525,  0.1084,  1.7287],
         [-0.1732,  1.3509, -0.4244,  0.0582,  0.0889,  1.7158],
         [-0.1672,  1.3338, -0.4425,  0.0640,  0.0688,  1.7013],
         [-0.1605,  1.3153, -0.4603,  0.0699,  0.0484,  1.6855],
         [-0.1531,  1.2957, -0.4779,  0.0760,  0.0276,  1.6684],
         [-0.1451,  1.2748, -0.4951,  0.0822,  0.0066,  1.6500],
         [-0.1363,  1.2528, -0.5120,  0.0886, -0.0146,  1.6304],
         [-0.1268,  1.2297, -0.5285,  0.0952, -0.0358,  1.6097],
         [-0.1165,  1.2057, -0.5445,  0.1021, -0.0570,  1.5880],
         [-0.1055,  1.1806, -0.5600,  0.1092, -0.0781,  1.5653],
         [-0.0936,  1.154

In [13]:
len(train_loader)

1291

In [14]:
len(test_loader)

323

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [19]:
sequence_length = 25  
prediction_length = 1  
input_channels = 6 
output_channels = 1

model = TSMixer(sequence_length, prediction_length, input_channels, output_channels)
criterion = criterion = nn.BCEWithLogitsLoss()   
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [16]:
# tsmixer accuracy , lstmden daha düşük , 

In [20]:
model.to(device)


TSMixer(
  (mixer_layers): Sequential(
    (0): MixerLayer(
      (time_mixing): TimeMixing(
        (norm): TimeBatchNorm2d(150, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (fc1): Linear(in_features=25, out_features=25, bias=True)
      )
      (feature_mixing): FeatureMixing(
        (norm_before): TimeBatchNorm2d(150, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (norm_after): Identity()
        (dropout): Dropout(p=0.1, inplace=False)
        (fc1): Linear(in_features=6, out_features=64, bias=True)
        (fc2): Linear(in_features=64, out_features=1, bias=True)
        (projection): Linear(in_features=6, out_features=1, bias=True)
      )
    )
  )
  (temporal_projection): Linear(in_features=25, out_features=1, bias=True)
)

In [21]:


num_epochs = 1
for epoch in range(num_epochs):
    for features, labels in train_loader:
        print(labels[epoch])
        print(features.shape)
        print(labels.shape)
        print(features.dtype)
        print(labels.dtype)
        

tensor(0)
torch.Size([32, 25, 6])
torch.Size([32])
torch.float32
torch.int64
tensor(1)
torch.Size([32, 25, 6])
torch.Size([32])
torch.float32
torch.int64
tensor(1)
torch.Size([32, 25, 6])
torch.Size([32])
torch.float32
torch.int64
tensor(1)
torch.Size([32, 25, 6])
torch.Size([32])
torch.float32
torch.int64
tensor(1)
torch.Size([32, 25, 6])
torch.Size([32])
torch.float32
torch.int64
tensor(1)
torch.Size([32, 25, 6])
torch.Size([32])
torch.float32
torch.int64
tensor(1)
torch.Size([32, 25, 6])
torch.Size([32])
torch.float32
torch.int64
tensor(1)
torch.Size([32, 25, 6])
torch.Size([32])
torch.float32
torch.int64
tensor(0)
torch.Size([32, 25, 6])
torch.Size([32])
torch.float32
torch.int64
tensor(1)
torch.Size([32, 25, 6])
torch.Size([32])
torch.float32
torch.int64
tensor(1)
torch.Size([32, 25, 6])
torch.Size([32])
torch.float32
torch.int64
tensor(1)
torch.Size([32, 25, 6])
torch.Size([32])
torch.float32
torch.int64
tensor(1)
torch.Size([32, 25, 6])
torch.Size([32])
torch.float32
torch.int64

KeyboardInterrupt: 

In [22]:
 
num_epochs = 30
for epoch in range(num_epochs):
   model.train()
   for features, labels in train_loader:
      features, labels = features.to(device), labels.to(device)
      optimizer.zero_grad()  # Gradyanları sıfırla
      outputs = model(features) 
      outputs = torch.squeeze(outputs)  # Fazladan boyutu kaldır

      labels = labels.float()
      
      #print(outputs.shape)
      #print(labels.shape)  # Modelden tahmin al
      loss = criterion(outputs, labels)  # Kayıp hesapla
      loss.backward()  # Geri yayılım
      optimizer.step()  # Ağı güncelle
   print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print("Training complete")   

Epoch [1/30], Loss: 0.3070
Epoch [2/30], Loss: 0.5079
Epoch [3/30], Loss: 0.4306
Epoch [4/30], Loss: 0.2730
Epoch [5/30], Loss: 0.2492
Epoch [6/30], Loss: 0.3204
Epoch [7/30], Loss: 0.5115
Epoch [8/30], Loss: 0.4417
Epoch [9/30], Loss: 0.5731
Epoch [10/30], Loss: 0.3044
Epoch [11/30], Loss: 0.2926
Epoch [12/30], Loss: 0.5859
Epoch [13/30], Loss: 0.4712
Epoch [14/30], Loss: 0.3333
Epoch [15/30], Loss: 0.3717
Epoch [16/30], Loss: 0.3156
Epoch [17/30], Loss: 0.7390
Epoch [18/30], Loss: 0.6033
Epoch [19/30], Loss: 0.4701
Epoch [20/30], Loss: 0.1227
Epoch [21/30], Loss: 0.2988
Epoch [22/30], Loss: 0.3505
Epoch [23/30], Loss: 0.3796
Epoch [24/30], Loss: 0.5826
Epoch [25/30], Loss: 0.2763
Epoch [26/30], Loss: 0.4610
Epoch [27/30], Loss: 0.4063
Epoch [28/30], Loss: 0.8217
Epoch [29/30], Loss: 0.2761
Epoch [30/30], Loss: 0.3778
Training complete


In [23]:
model.eval()
test_loss = 0.0  
correct_predictions = 0 
total_predictions = 0  
with torch.no_grad():
    for features, labels in test_loader:
        
        features, labels = features.to(device), labels.to(device)
        outputs = model(features)
        # Fazladan boyut varsa kaldır
        outputs = torch.squeeze(outputs)
        labels = labels.float()
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        
        # Tahminler: outputs > 0.5 (Sigmoid aktivasyonu sonucu) # buna gerek yok , normalde denemek için
        predicted = (outputs > 0.5).float()
        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)

average_test_loss = test_loss / len(test_loader)
accuracy = correct_predictions / total_predictions

print(f'Test Loss: {average_test_loss:.4f}, Accuracy: {accuracy:.4f}')

Test Loss: 0.2951, Accuracy: 0.8844


In [25]:
# Test moduna geç
model.eval()
test_loss = 0.0 
correct_predictions = 0  
total_predictions = 0 
# Test sırasında gradient hesaplanmayacak
with torch.no_grad():
    for features, labels in test_loader:
        # Verileri GPU'ya gönder
        features, labels = features.to(device), labels.to(device)
        outputs = model(features)
        outputs = torch.squeeze(outputs)
        
        # Label'ları float'a çevir (BCEWithLogitsLoss için)
        labels = labels.float()
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        
        # Tahminler: Sigmoid gerekmez, direkt olarak 0 ile 1 karşılaştırması yapılır #bcelogits kullandığımız için
        predicted = (outputs > 0).float()
        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)

average_test_loss = test_loss / len(test_loader)
accuracy = correct_predictions / total_predictions

print(f'Test Loss: {average_test_loss:.4f}, Accuracy: {accuracy:.4f}')


Test Loss: 0.2951, Accuracy: 0.8824


In [24]:
 
torch.save(model.state_dict(), 'model_weights_ts_mixer_output_23_09.pth')


# Modeli tanımla (aynı yapı ile)
model = TSMixer(sequence_length, prediction_length, input_channels, output_channels)
model.load_state_dict(torch.load('model_weights.pth'))
model.to(device)  # Modeli uygun cihaza gönder


on katmanı sigmoide çevirebilir miyiz ? 
ChatGPT:
ChatGPT
Evet, son katmanı sigmoid aktivasyon fonksiyonuna çevirebilirsiniz. Eğer iki sınıflı bir problem üzerinde çalışıyorsanız ve çıkış olarak tek bir değer (0 ile 1 arasında) almak istiyorsanız, son katmanın çıkışını sigmoid aktivasyonu ile uygulamak mantıklıdır.

Bunu yapmak için son katmanı şu şekilde güncelleyebilirsiniz:

python
Kodu kopyala
self.temporal_projection = Linear(in_features=25, out_features=1, bias=True)
Sonrasında modelin çıkışına sigmoid aktivasyonunu eklemeniz gerekir:

python
Kodu kopyala
import torch.nn.functional as F

# Forward pass içinde
outputs = model(x)
outputs = torch.sigmoid(outputs)
Bu durumda, kayıp fonksiyonu olarak BCELoss (Binary Cross-Entropy Loss) kullanmalısınız. Yani, hedef değerlerinizin de 0 veya 1 şeklinde olmasına dikkat edin.

Özetle, son katmanı sigmoid aktivasyonu ile kullanmak için sadece çıkışı bu şekilde değiştirmelisiniz.






Şunu dedin:
bunu nerede değiştirebilirim tsmixer için ? :self.temporal_projection = Linear(in_features=25, out_features=1, bias=True)
ChatGPT:
ChatGPT
TSMixer modelinde son katmanı değiştirmek için, genellikle modelin forward metodunun son aşamasındaki çıkış katmanında bir lineer katman kullanılır. self.temporal_projection veya benzeri bir isimle tanımlanmış bir lineer katman son katman olarak görev alıyor olabilir. Bu katmanı değiştirmek için şu adımları izleyebilirsiniz:

TSMixer modelini bulun: Modelin __init__ fonksiyonunda, self.temporal_projection benzeri bir lineer katmanın tanımlı olduğu yeri bulun.

Son katmanı güncelleyin: Eğer out_features kısmı iki sınıflı bir problem içinse, bunu 1 yaparak sigmoid aktivasyonu ile çalışacak şekilde güncelleyebilirsiniz. Örneğin:

python
Kodu kopyala
self.temporal_projection = nn.Linear(in_features=25, out_features=1, bias=True)
Sigmoid aktivasyonunu ekleyin: Modelin forward fonksiyonunda, self.temporal_projection kısmından hemen sonra bir sigmoid aktivasyonu ekleyin. Örneğin:

python
Kodu kopyala
def forward(self, x):
    # Diğer katmanlar...
    x = self.temporal_projection(x)
    x = torch.sigmoid(x)  # Sigmoid aktivasyonunu burada ekleyin
    return x
Örnek TSMixer Yapısı:
python
Kodu kopyala
class TSMixer(nn.Module):
    def __init__(self, sequence_length, prediction_length, input_channels, output_channels):
        super(TSMixer, self).__init__()
        self.mixer_layers = nn.ModuleList([...])  # Diğer katmanlar
        self.temporal_projection = nn.Linear(in_features=25, out_features=1, bias=True)  # Değiştirilen son katman

    def forward(self, x):
        # TSMixer'de diğer katmanlar ve işlemler
        x = self.temporal_projection(x)
        x = torch.sigmoid(x)  # Sigmoid aktivasyon
        return x
Bu güncellemeleri yaptıktan sonra, model iki sınıflı problemde sigmoid çıkış verecek ve BCELoss ile uyumlu olacaktır.

In [61]:
model

TSMixer(
  (mixer_layers): Sequential(
    (0): MixerLayer(
      (time_mixing): TimeMixing(
        (norm): TimeBatchNorm2d(150, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (fc1): Linear(in_features=25, out_features=25, bias=True)
      )
      (feature_mixing): FeatureMixing(
        (norm_before): TimeBatchNorm2d(150, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (norm_after): Identity()
        (dropout): Dropout(p=0.1, inplace=False)
        (fc1): Linear(in_features=6, out_features=64, bias=True)
        (fc2): Linear(in_features=64, out_features=1, bias=True)
        (projection): Linear(in_features=6, out_features=1, bias=True)
      )
    )
  )
  (temporal_projection): Linear(in_features=25, out_features=1, bias=True)
)